In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/waterscarcity/visualization_outputUp/Evap/A19810224.png
/kaggle/input/waterscarcity/visualization_outputUp/Evap/A19810408.png
/kaggle/input/waterscarcity/visualization_outputUp/Evap/A19790527.png
/kaggle/input/waterscarcity/visualization_outputUp/Evap/A19790902.png
/kaggle/input/waterscarcity/visualization_outputUp/Evap/A19790315.png
/kaggle/input/waterscarcity/visualization_outputUp/Evap/A19791024.png
/kaggle/input/waterscarcity/visualization_outputUp/Evap/A19810313.png
/kaggle/input/waterscarcity/visualization_outputUp/Evap/A19800319.png
/kaggle/input/waterscarcity/visualization_outputUp/Evap/A19811004.png
/kaggle/input/waterscarcity/visualization_outputUp/Evap/A19800923.png
/kaggle/input/waterscarcity/visualization_outputUp/Evap/A19810811.png
/kaggle/input/waterscarcity/visualization_outputUp/Evap/A19791205.png
/kaggle/input/waterscarcity/visualization_outputUp/Evap/A19801107.png
/kaggle/input/waterscarcity/visualization_outputUp/Evap/A19801027.png
/kaggle/input/waters

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.metrics import classification_report, roc_curve, RocCurveDisplay
from PIL import Image
from tqdm import tqdm

2025-04-27 22:33:31.903039: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745793211.928103    1242 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745793211.935556    1242 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
FEATURES = ['Evap', 'Rainf', 'RootMoist', 'SoilM_0_10cm', 'TVeg']
BASE_PATH = "/kaggle/input/water-scarcity/visualization_outputUp"  # Update this path to your data location
IMG_SIZE = (176//2, 260//2)  # Adjust based on your actual image dimensions
BATCH_SIZE = 32  # Larger batches for better GPU utilization
EPOCHS = 50
RANDOM_SEED = 42

In [4]:
try:
    import nvidia_smi
    NVIDIA_SMI_AVAILABLE = True
except ImportError:
    NVIDIA_SMI_AVAILABLE = False
    print("nvidia-smi not available. GPU monitoring disabled.")


nvidia-smi not available. GPU monitoring disabled.


In [5]:
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [6]:
def setup_gpu():
    # Vérifier les GPU disponibles
    gpus = tf.config.list_physical_devices('GPU')
    print(f"GPU disponibles: {len(gpus)}")
    for gpu in gpus:
        print(f" - {gpu}")
    
    if len(gpus) == 0:
        print("Aucun GPU détecté!")
        return None
    
    # Configurer la croissance de mémoire pour éviter d'allouer toute la mémoire immédiatement
    for gpu in gpus:
        try:
            tf.config.experimental.set_memory_growth(gpu, True)
            print(f"Memory growth activé pour {gpu}")
        except RuntimeError as e:
            print(f"Erreur lors de la configuration de la mémoire pour {gpu}: {e}")
    
    # Activer la journalisation des opérations de distribution si nécessaire
    # os.environ["TF_CPP_MIN_LOG_LEVEL"] = "0"
    # tf.debugging.set_log_device_placement(True)
    
    # Créer une stratégie de distribution MirroredStrategy pour utiliser tous les GPU
    strategy = tf.distribute.MirroredStrategy()
    print(f"Nombre de périphériques dans la stratégie: {strategy.num_replicas_in_sync}")
    
    return strategy

In [7]:
import tensorflow as tf

# List all physical GPU devices
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Print which device is used
if tf.test.gpu_device_name():
    print("TensorFlow is using GPU:", tf.test.gpu_device_name())
else:
    print("TensorFlow is NOT using the GPU.")


Num GPUs Available:  2
TensorFlow is using GPU: /device:GPU:0


I0000 00:00:1745793215.748914    1242 gpu_device.cc:2022] Created device /device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1745793215.749647    1242 gpu_device.cc:2022] Created device /device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
I0000 00:00:1745793215.752414    1242 gpu_device.cc:2022] Created device /device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1745793215.752740    1242 gpu_device.cc:2022] Created device /device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [8]:
def load_images_from_folder(folder_path, img_size):
    """Load and preprocess images from a folder"""
    images = []
    filenames = []
    
    print(f"Loading images from {folder_path}...")
    for filename in tqdm(sorted(os.listdir(folder_path))):
        if filename.endswith((".png", ".jpg", ".jpeg")):
            img_path = os.path.join(folder_path, filename)
            img = Image.open(img_path).convert('L')  # grayscale
            img = img.resize((img_size[1], img_size[0]))  # (width, height)
            images.append(np.array(img) / 255.0)  # Normalize
            filenames.append(filename)
            
    return np.array(images), filenames


In [9]:
def build_autoencoder(input_shape):
    inputs = layers.Input(shape=input_shape)

    # Encoder
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D(2, padding='same')(x)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    encoded = layers.MaxPooling2D(2, padding='same')(x)

    # Decoder
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(encoded)
    x = layers.UpSampling2D(2)(x)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
    x = layers.UpSampling2D(2)(x)
    decoded = layers.Conv2D(1, 3, activation='sigmoid', padding='same')(x)

    autoencoder = models.Model(inputs, decoded)
    return autoencoder


In [10]:
def spatial_attention_block(input_tensor):
    """Add spatial attention to focus on important regions"""
    # Use Lambda layers to wrap TensorFlow operations
    avg_pool = layers.Lambda(lambda x: tf.reduce_mean(x, axis=-1, keepdims=True))(input_tensor)
    max_pool = layers.Lambda(lambda x: tf.reduce_max(x, axis=-1, keepdims=True))(input_tensor)
    concat = layers.Concatenate()([avg_pool, max_pool])
    attention = layers.Conv2D(1, 7, activation='sigmoid', padding='same')(concat)
    return layers.Multiply()([input_tensor, attention])

In [11]:
def build_attention_autoencoder(input_shape):
    """Autoencoder with spatial attention mechanism that preserves dimensions exactly"""
    inputs = layers.Input(shape=input_shape)
    
    # Store original dimensions
    h, w, c = input_shape
    
    # Encoder with attention
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    x = spatial_attention_block(x)
    x = layers.MaxPooling2D(2, padding='same')(x)
    
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = spatial_attention_block(x)
    x = layers.MaxPooling2D(2, padding='same')(x)
    
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    encoded = layers.MaxPooling2D(2, padding='same')(x)
    
    # Decoder
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(encoded)
    x = layers.UpSampling2D(2)(x)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.UpSampling2D(2)(x)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
    x = layers.UpSampling2D(2)(x)
    
    # Final layer with cropping/padding to ensure exact dimension match
    x = layers.Conv2D(1, 3, activation='sigmoid', padding='same')(x)
    
    # Add a Lambda layer to resize to exact input dimensions
    decoded = layers.Lambda(
        lambda x: tf.image.resize(x, [h, w])
    )(x)
    
    return models.Model(inputs, decoded)

sta3melt autoencoder hna (convolutional auto encoder CAU) khater maandich labels donc sta3meltou bch yreconstructy el input images mta3y, ken famma noise yreducih w bch yaaml highlit lel key spacial patterns, bch baad najjem naaml analyse lel tsawer

In [12]:
def visualize_reconstruction(original, reconstructed, n=5):
    """Visualize original and reconstructed images side by side"""
    plt.figure(figsize=(15, 5))
    for i in range(min(n, len(original))):
        # Original
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(original[i].squeeze(), cmap='gray')
        plt.title("Original")
        plt.axis("off")

        # Reconstructed
        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(reconstructed[i].squeeze(), cmap='gray')
        plt.title("Reconstructed")
        plt.axis("off")
    plt.tight_layout()
    plt.show()

In [13]:
def detect_anomalies(autoencoder, images, threshold_quantile=0.95):
    """Detect anomalies based on reconstruction error"""
    print("Detecting anomalies...")
    reconstructions = autoencoder.predict(images, verbose=0)
    # Calculate per-pixel error
    pixel_errors = np.square(images - reconstructions)
    # Calculate per-image error
    image_errors = np.mean(pixel_errors, axis=(1, 2, 3))
    # Determine threshold
    threshold = np.quantile(image_errors, threshold_quantile)
    return pixel_errors, image_errors, threshold


In [14]:
def visualize_anomalies(images, pixel_errors, image_errors, threshold, n=5):
    """Visualize images with anomaly heatmaps"""
    plt.figure(figsize=(20, 10))
    
    # Select n images with highest error
    highest_error_indices = np.argsort(image_errors)[-n:]
    
    for i, idx in enumerate(highest_error_indices):
        # Original image
        plt.subplot(2, n, i+1)
        plt.imshow(images[idx].squeeze(), cmap='gray')
        plt.title(f"Original (idx: {idx})")
        plt.axis('off')
        
        # Anomaly heatmap
        plt.subplot(2, n, i+1+n)
        plt.imshow(np.mean(pixel_errors[idx], axis=-1), cmap='hot', alpha=0.7)
        plt.colorbar()
        plt.title(f"Error: {image_errors[idx]:.4f}")
        plt.axis('off')
    
    plt.suptitle(f"Threshold: {threshold:.4f}")
    plt.tight_layout()
    plt.show()

In [15]:
def cluster_anomalies(features, n_clusters=3, visualize=True):
    """Cluster features and visualize results using t-SNE"""
    print(f"Clustering data into {n_clusters} clusters...")
    
    # Flatten features if needed
    if len(features.shape) > 2:
        features_flat = features.reshape(features.shape[0], -1)
    else:
        features_flat = features
    
    # Cluster
    kmeans = KMeans(n_clusters=n_clusters, random_state=RANDOM_SEED)
    clusters = kmeans.fit_predict(features_flat)
    
    # Visualization with t-SNE
    if visualize and features_flat.shape[1] > 2:
        print("Performing t-SNE dimensionality reduction...")
        tsne = TSNE(n_components=2, random_state=RANDOM_SEED)
        reduced = tsne.fit_transform(features_flat)
        
        plt.figure(figsize=(10, 6))
        scatter = plt.scatter(reduced[:, 0], reduced[:, 1], c=clusters, 
                             cmap='viridis', alpha=0.7)
        plt.colorbar(scatter, label='Cluster')
        plt.title('t-SNE Visualization of Drought Patterns')
        plt.xlabel('t-SNE 1')
        plt.ylabel('t-SNE 2')
        plt.show()
    
    return clusters


In [16]:
def visualize_clusters(X, anomalous_indices, clusters):
    """Visualize representative samples from each cluster"""
    n_clusters = len(np.unique(clusters))
    
    plt.figure(figsize=(15, 5))
    for cluster_id in range(n_clusters):
        # Get cluster samples
        cluster_samples_idx = np.where(clusters == cluster_id)[0]
        cluster_samples = X[anomalous_indices][cluster_samples_idx]
        n_samples = len(cluster_samples)
        
        # Plot up to 3 samples per cluster
        for i, sample_idx in enumerate(cluster_samples_idx[:3]):
            plt.subplot(n_clusters, 3, cluster_id * 3 + i + 1)
            plt.imshow(X[anomalous_indices[sample_idx]].squeeze(), cmap='gray')
            plt.title(f"Cluster {cluster_id}\n({n_samples} samples)")
            plt.axis('off')
    
    plt.tight_layout()
    plt.show()

In [17]:
def approximate_ndvi_from_grayscale(images):
    """Approximate NDVI values from grayscale (for demonstration)"""
    # This is just a placeholder since we don't have true NDVI data
    # In reality, NDVI requires NIR and Red bands
    print("Approximating NDVI values from grayscale...")
    pseudo_ndvi = 2 * images.squeeze() - 1  # Scale [0,1] to [-1,1]
    return pseudo_ndvi

def validate_with_ndvi(autoencoder, images, ndvi_maps):
    """Validate anomaly detection against NDVI values"""
    print("Validating with NDVI...")
    # Get reconstruction errors
    reconstructions = autoencoder.predict(images, verbose=0)
    errors = np.mean(np.square(images - reconstructions), axis=(1, 2, 3))
    
    # Calculate mean NDVI per image
    mean_ndvi = ndvi_maps.mean(axis=(1, 2))
    
    # Calculate correlation
    correlation = np.corrcoef(mean_ndvi, errors)[0, 1]
    
    # Visualization
    plt.figure(figsize=(10, 5))
    plt.scatter(mean_ndvi, errors, alpha=0.6)
    plt.xlabel('Mean NDVI')
    plt.ylabel('Reconstruction Error')
    plt.title(f'Correlation: {correlation:.2f}')
    plt.grid()
    plt.show()
    
    return correlation

In [18]:
def overlay_validation(images, ndvi, pixel_errors, img_indices):
    """Side-by-side comparison of original image, NDVI and anomaly detection"""
    for img_idx in img_indices:
        plt.figure(figsize=(15, 5))
        
        # Original Image
        plt.subplot(1, 3, 1)
        plt.imshow(images[img_idx].squeeze(), cmap='gray')
        plt.title("Original Image")
        plt.axis('off')
        
        # NDVI Map
        plt.subplot(1, 3, 2)
        plt.imshow(ndvi[img_idx], cmap='RdYlGn', vmin=-1, vmax=1)
        plt.colorbar()
        plt.title(f"NDVI Map (Mean: {ndvi[img_idx].mean():.2f})")
        plt.axis('off')
        
        # Anomaly Map
        plt.subplot(1, 3, 3)
        plt.imshow(np.mean(pixel_errors[img_idx], axis=-1), cmap='hot')
        plt.colorbar()
        plt.title("Reconstruction Error")
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()


In [19]:
def evaluate_with_ndvi_threshold(image_errors, ndvi_maps, ndvi_threshold=0.3):
    """Evaluate anomaly detection using NDVI-based 'ground truth'"""
    print(f"Evaluating using NDVI threshold of {ndvi_threshold}...")
    
    # Create NDVI-based labels (1=drought, 0=healthy)
    ndvi_mean = ndvi_maps.mean(axis=(1, 2))
    ndvi_labels = (ndvi_mean < ndvi_threshold).astype(int)
    
    # Find optimal error threshold (Youden's J statistic)
    fpr, tpr, thresholds = roc_curve(ndvi_labels, image_errors)
    optimal_idx = np.argmax(tpr - fpr)
    optimal_threshold = thresholds[optimal_idx]
    
    # Metrics
    predictions = (image_errors > optimal_threshold).astype(int)
    print(classification_report(ndvi_labels, predictions))
    print(f"Optimal threshold: {optimal_threshold:.4f}")
    
    # Plot ROC curve
    RocCurveDisplay.from_predictions(ndvi_labels, image_errors)
    plt.show()
    
    return optimal_threshold

In [20]:
def process_new_image(img_path, autoencoder, threshold):
    """Process a new image for drought detection"""
    img = Image.open(img_path).convert('L')
    img = img.resize((IMG_SIZE[1], IMG_SIZE[0]))  # width, height
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=(0, -1))
    
    reconstruction = autoencoder.predict(img_array, verbose=0)
    error = np.mean((img_array - reconstruction) ** 2)
    
    is_drought = error > threshold
    
    # Visualization
    plt.figure(figsize=(15, 5))
    
    plt.subplot(1, 3, 1)
    plt.imshow(img_array[0].squeeze(), cmap='gray')
    plt.title("Original Image")
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(reconstruction[0].squeeze(), cmap='gray')
    plt.title("Reconstructed Image")
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    plt.imshow(np.square(img_array[0] - reconstruction[0]).squeeze(), cmap='hot')
    plt.colorbar()
    plt.title(f"Error: {error:.4f}\nThreshold: {threshold:.4f}")
    plt.axis('off')
    
    plt.suptitle(f"Drought Detected: {'YES' if is_drought else 'NO'}", 
                fontsize=16, color='red' if is_drought else 'green')
    plt.tight_layout()
    plt.show()
    
    return is_drought, error


In [21]:
def find_optimal_clusters(features, max_k=10):
    """Find optimal number of clusters using elbow method"""
    print("Finding optimal number of clusters...")
    distortions = []
    K_range = range(1, max_k + 1)
    for k in tqdm(K_range):
        kmeans_model = KMeans(n_clusters=k, random_state=RANDOM_SEED)
        kmeans_model.fit(features)
        distortions.append(kmeans_model.inertia_)

    plt.figure(figsize=(10, 6))
    plt.plot(K_range, distortions, 'bx-')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Distortion (Inertia)')
    plt.title('Elbow Method for Optimal k')
    plt.grid()
    plt.show()
    
    # Calculate rate of change to find elbow point
    changes = np.diff(distortions)
    changes_ratio = changes[1:] / changes[:-1]
    elbow_point = np.argmin(changes_ratio) + 2  # +2 because of offset
    
    print(f"Suggested optimal number of clusters: {elbow_point}")
    return elbow_point


In [22]:
def analyze_drought_patterns(feature_name):
    """Complete drought pattern analysis pipeline for a single feature"""
    print(f"\n{'='*20} Processing {feature_name} {'='*20}")
    
    # 1. Setup
    setup_gpu()
    
    # 2. Load Data
    folder_path = os.path.join(BASE_PATH, feature_name)
    X, filenames = load_images_from_folder(folder_path, IMG_SIZE)
    X = np.expand_dims(X, -1)  # Add channel dimension
    print(f"Loaded {X.shape[0]} images for {feature_name}")
    
    # 3. Split data
    X_train, X_test = train_test_split(X, test_size=0.2, random_state=RANDOM_SEED)
    print(f"Training set: {X_train.shape[0]}, Test set: {X_test.shape[0]}")
    
    # 4. Build and train model
    print("Building and training autoencoder...")
    autoencoder = build_attention_autoencoder((*IMG_SIZE, 1))
    autoencoder.compile(optimizer='adam', loss='mse')
    autoencoder.summary()
    
    # Train with callbacks for early stopping and model checkpointing
    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3)
    ]
    
    history = autoencoder.fit(
        X_train, X_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        shuffle=True,
        validation_data=(X_test, X_test),
        callbacks=callbacks,
        verbose=1
    )
    
    # 5. Visualize training history
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    if 'lr' in history.history:
        plt.subplot(1, 2, 2)
        plt.plot(history.history['lr'])
        plt.title('Learning Rate')
        plt.xlabel('Epoch')
        plt.ylabel('Learning Rate')
    
    plt.tight_layout()
    plt.show()
    
    # 6. Visualize some reconstructions
    print("Generating reconstructions...")
    decoded_imgs = autoencoder.predict(X_test[:10], verbose=0)
    visualize_reconstruction(X_test[:10], decoded_imgs)
    
    # 7. Extract latent features (encoder output)
    print("Extracting latent features...")
    encoder = models.Model(autoencoder.input, autoencoder.layers[4].output)
    encoded_features = encoder.predict(X, verbose=0)
    
    # Get flattened features for clustering
    features_flat = encoded_features.reshape(encoded_features.shape[0], -1)
    
    # 8. Detect anomalies
    pixel_errors, image_errors, threshold = detect_anomalies(autoencoder, X)
    visualize_anomalies(X, pixel_errors, image_errors, threshold)
    
    # 9. Get anomalous indices
    anomalous_indices = np.where(image_errors > threshold)[0]
    print(f"Found {len(anomalous_indices)} potential drought areas")
    
    # 10. Find optimal number of clusters (if we have enough anomalies)
    if len(anomalous_indices) >= 10:
        optimal_k = find_optimal_clusters(features_flat[anomalous_indices])
    else:
        optimal_k = min(len(anomalous_indices), 3)
        print(f"Too few anomalies, using k={optimal_k}")
    
    # 11. Cluster anomalies
    anomalous_features = features_flat[anomalous_indices]
    clusters = cluster_anomalies(anomalous_features, n_clusters=optimal_k)
    
    # 12. Visualize clusters
    visualize_clusters(X, anomalous_indices, clusters)
    
    # 13. Optional: NDVI validation
    print("Generating approximate NDVI for validation...")
    # This is a placeholder since we don't have true NDVI data
    pseudo_ndvi = approximate_ndvi_from_grayscale(X)
    
    # Validate anomaly detection against NDVI
    correlation = validate_with_ndvi(autoencoder, X, pseudo_ndvi)
    
    # Show side-by-side comparisons for a few samples
    overlay_validation(X, pseudo_ndvi, pixel_errors, 
                      anomalous_indices[:3])  # Show first 3 anomalies
    
    # 14. Evaluate against NDVI threshold
    eval_threshold = evaluate_with_ndvi_threshold(image_errors, pseudo_ndvi)
    
    # 15. Return the model and threshold for later use
    return autoencoder, threshold, eval_threshold, anomalous_indices, clusters


In [23]:
def find_optimal_clusters(features, max_k=10):
    """Find optimal number of clusters using elbow method"""
    print("Finding optimal number of clusters...")
    distortions = []
    K_range = range(1, max_k + 1)
    for k in tqdm(K_range):
        kmeans_model = KMeans(n_clusters=k, random_state=RANDOM_SEED)
        kmeans_model.fit(features)
        distortions.append(kmeans_model.inertia_)

    plt.figure(figsize=(10, 6))
    plt.plot(K_range, distortions, 'bx-')
    plt.xlabel('Number of clusters (k)')
    plt.ylabel('Distortion (Inertia)')
    plt.title('Elbow Method for Optimal k')
    plt.grid()
    plt.show()
    
    # Calculate rate of change to find elbow point
    changes = np.diff(distortions)
    changes_ratio = changes[1:] / changes[:-1]
    elbow_point = np.argmin(changes_ratio) + 2  # +2 because of offset
    
    print(f"Suggested optimal number of clusters: {elbow_point}")
    return elbow_point

In [24]:
import gc
def analyze_drought_patterns(feature_name):
    """Drought pattern analysis with memory management"""
    print(f"\n{'='*20} Processing {feature_name} {'='*20}")
    
    # 1. Load Data (smaller batch)
    folder_path = os.path.join(BASE_PATH, feature_name)
    X, filenames = load_images_from_folder(folder_path, IMG_SIZE)
    X = np.expand_dims(X, -1)  # Add channel dimension
    print(f"Loaded {X.shape[0]} images for {feature_name}")
    
    # Print memory usage information
    print(f"Dataset shape: {X.shape}, Memory usage: {X.nbytes / (1024**2):.2f} MB")
    
    # 2. Split data
    X_train, X_test = train_test_split(X, test_size=0.2, random_state=RANDOM_SEED)
    
    # 3. Build a simpler model (reduce complexity)
    print("Building simpler autoencoder...")
    input_shape = (*IMG_SIZE, 1)
    
    # Simpler autoencoder model
    inputs = tf.keras.layers.Input(shape=input_shape)
    
    # Encoder (reduced filters)
    x = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same')(inputs)
    x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
    x = tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
    encoded = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
    
    # Decoder (matching encoder)
    x = tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
    x = tf.keras.layers.UpSampling2D((2, 2))(x)
    x = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = tf.keras.layers.UpSampling2D((2, 2))(x)
    decoded = tf.keras.layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
    
    autoencoder = tf.keras.models.Model(inputs, decoded)
    autoencoder.compile(optimizer='adam', loss='mse')
    autoencoder.summary()
    
    # 4. Train with smaller batch size
    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3)
    ]
    
    # Smaller batch size
    batch_size = 4  # Try a much smaller batch
    
    try:
        history = autoencoder.fit(
            X_train, X_train,
            epochs=EPOCHS,
            batch_size=batch_size,
            shuffle=True,
            validation_data=(X_test, X_test),
            callbacks=callbacks,
            verbose=1
        )
        print("Training completed successfully")
    except Exception as e:
        print(f"Error during training: {e}")
        return None, None, None, None, None
    
    # Clear memory after training
    tf.keras.backend.clear_session()
    gc.collect()
    
    # 5. Detect anomalies in batches
    print("Detecting anomalies in batches...")
    batch_size = 4
    n_samples = X.shape[0]
    reconstructed_chunks = []
    
    # Process in chunks to avoid memory issues
    for i in range(0, n_samples, batch_size):
        end_idx = min(i + batch_size, n_samples)
        chunk = X[i:end_idx]
        reconstructed_chunk = autoencoder.predict(chunk, verbose=0)
        reconstructed_chunks.append(reconstructed_chunk)
    
    reconstructed_images = np.concatenate(reconstructed_chunks, axis=0)
    pixel_errors = np.abs(X - reconstructed_images)
    image_errors = np.mean(pixel_errors, axis=(1, 2, 3))
    threshold = np.mean(image_errors) + 2 * np.std(image_errors)
    
    # Get anomalous indices
    anomalous_indices = np.where(image_errors > threshold)[0]
    
    # Simple clustering (reduced complexity)
    n_clusters = min(3, len(anomalous_indices))
    clusters = np.zeros(len(anomalous_indices), dtype=int)
    
    # Return results
    eval_threshold = threshold  # Simplified
    return autoencoder, threshold, eval_threshold, anomalous_indices, clusters

In [25]:
def process_new_image(img_path, autoencoder, threshold):
    img = Image.open(img_path).convert('L')
    img = img.resize(IMG_SIZE[::-1])
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=(0, -1))
    
    reconstruction = autoencoder.predict(img_array)
    error = np.mean((img_array - reconstruction) ** 2)
    
    if error > threshold:
        print("Drought-prone area detected!")
        return True, error
    return False, error

In [26]:
def analyze_drought_patterns(feature_name):
    """Complete drought pattern analysis pipeline for a single feature"""
    print(f"\n{'='*20} Processing {feature_name} {'='*20}")
    
    # 1. Setup
    setup_gpu()
    
    # 2. Load Data
    folder_path = os.path.join(BASE_PATH, feature_name)
    X, filenames = load_images_from_folder(folder_path, IMG_SIZE)
    X = np.expand_dims(X, -1)  # Add channel dimension
    print(f"Loaded {X.shape[0]} images for {feature_name}")
    
    # 3. Split data
    X_train, X_test = train_test_split(X, test_size=0.2, random_state=RANDOM_SEED)
    print(f"Training set: {X_train.shape[0]}, Test set: {X_test.shape[0]}")
    
    # 4. Build and train model
    print("Building and training autoencoder...")
    autoencoder = build_attention_autoencoder((*IMG_SIZE, 1))
    autoencoder.compile(optimizer='adam', loss='mse')
    autoencoder.summary()
    
    # Train with callbacks for early stopping and model checkpointing
    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3)
    ]
    
    history = autoencoder.fit(
        X_train, X_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        shuffle=True,
        validation_data=(X_test, X_test),
        callbacks=callbacks,
        verbose=1
    )
    
    # 5. Visualize training history
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    if 'lr' in history.history:
        plt.subplot(1, 2, 2)
        plt.plot(history.history['lr'])
        plt.title('Learning Rate')
        plt.xlabel('Epoch')
        plt.ylabel('Learning Rate')
    
    plt.tight_layout()
    plt.show()
    
    # 6. Visualize some reconstructions
    print("Generating reconstructions...")
    decoded_imgs = autoencoder.predict(X_test[:10], verbose=0)
    visualize_reconstruction(X_test[:10], decoded_imgs)
    
    # 7. Extract latent features (encoder output)
    print("Extracting latent features...")
    encoder = models.Model(autoencoder.input, autoencoder.layers[4].output)
    encoded_features = encoder.predict(X, verbose=0)
    
    # Get flattened features for clustering
    features_flat = encoded_features.reshape(encoded_features.shape[0], -1)
    
    # 8. Detect anomalies
    pixel_errors, image_errors, threshold = detect_anomalies(autoencoder, X)
    visualize_anomalies(X, pixel_errors, image_errors, threshold)
    
    # 9. Get anomalous indices
    anomalous_indices = np.where(image_errors > threshold)[0]
    print(f"Found {len(anomalous_indices)} potential drought areas")
    
    # 10. Find optimal number of clusters (if we have enough anomalies)
    if len(anomalous_indices) >= 10:
        optimal_k = find_optimal_clusters(features_flat[anomalous_indices])
    else:
        optimal_k = min(len(anomalous_indices), 3)
        print(f"Too few anomalies, using k={optimal_k}")
    
    # 11. Cluster anomalies
    anomalous_features = features_flat[anomalous_indices]
    clusters = cluster_anomalies(anomalous_features, n_clusters=optimal_k)
    
    # 12. Visualize clusters
    visualize_clusters(X, anomalous_indices, clusters)
    
    # 13. Optional: NDVI validation
    print("Generating approximate NDVI for validation...")
    # This is a placeholder since we don't have true NDVI data
    pseudo_ndvi = approximate_ndvi_from_grayscale(X)
    
    # Validate anomaly detection against NDVI
    correlation = validate_with_ndvi(autoencoder, X, pseudo_ndvi)
    
    # Show side-by-side comparisons for a few samples
    overlay_validation(X, pseudo_ndvi, pixel_errors, 
                      anomalous_indices[:3])  # Show first 3 anomalies
    
    # 14. Evaluate against NDVI threshold
    eval_threshold = evaluate_with_ndvi_threshold(image_errors, pseudo_ndvi)
    
    # 15. Return the model and threshold for later use
    return autoencoder, threshold, eval_threshold, anomalous_indices, clusters


In [27]:
from datetime import datetime
import numpy as np

def date_to_features(date_obj):
    """
    Convert a date into cyclical time features for the model.
    
    Args:
        date_obj: Either a datetime object or string in format '%Y-%m-%d'
    
    Returns:
        np.array: [year, month, day, day_of_year, month_sin, month_cos, day_sin, day_cos]
    """
    if isinstance(date_obj, str):
        date = datetime.strptime(date_obj, '%Y-%m-%d')
    else:
        date = date_obj
    
    year = date.year
    month = date.month
    day = date.day
    day_of_year = date.timetuple().tm_yday
    
    # Cyclical features for seasonality
    month_sin = np.sin(2 * np.pi * month / 12)
    month_cos = np.cos(2 * np.pi * month / 12)
    day_sin = np.sin(2 * np.pi * day_of_year / 365)
    day_cos = np.cos(2 * np.pi * day_of_year / 365)
    
    return np.array([year, month, day, day_of_year, month_sin, month_cos, day_sin, day_cos])

In [28]:
import numpy as np
import gc

def create_sequences_with_dates(data, dates, seq_length=30, pred_steps=7, batch_size=100):
    """
    Create time-series sequences with date features.
    
    Args:
        data: Numpy array of shape (n_samples, height, width, n_features)
        dates: List of datetime objects/strings matching data
        seq_length: Input sequence length
        pred_steps: Steps to predict
        batch_size: Memory optimization
    
    Returns:
        X: Input sequences (n_sequences, seq_length, h, w, n_features)
        y: Target sequences (n_sequences, pred_steps, h, w, n_features)
        date_features: Date features for each sequence (n_sequences, 8)
    """
    total_sequences = len(data) - seq_length - pred_steps + 1
    
    # Preallocate arrays
    X = np.zeros((total_sequences, seq_length, *data.shape[1:]), dtype=np.float32)
    y = np.zeros((total_sequences, pred_steps, *data.shape[1:]), dtype=np.float32)
    date_features = np.zeros((total_sequences, 8), dtype=np.float32)
    
    for start_idx in range(0, total_sequences, batch_size):
        end_idx = min(start_idx + batch_size, total_sequences)
        
        for i in range(start_idx, end_idx):
            X[i] = data[i:i+seq_length]
            y[i] = data[i+seq_length:i+seq_length+pred_steps]
            date_features[i] = date_to_features(dates[i+seq_length-1])
        
        if (end_idx - start_idx) % (batch_size * 5) == 0:
            gc.collect()
    
    return X, y, date_features

In [29]:
from tensorflow.keras import layers

def spatial_attention_block(input_tensor):
    """
    Spatial attention mechanism for focusing on important regions.
    
    Args:
        input_tensor: Input feature map
    
    Returns:
        Tensor: Feature map with attention weights applied
    """
    # Channel average and max pooling
    avg_pool = layers.Lambda(lambda x: tf.reduce_mean(x, axis=-1, keepdims=True))(input_tensor)
    max_pool = layers.Lambda(lambda x: tf.reduce_max(x, axis=-1, keepdims=True))(input_tensor)
    
    # Concatenate and apply conv
    concat = layers.Concatenate()([avg_pool, max_pool])
    attention = layers.Conv2D(1, 7, activation='sigmoid', padding='same')(concat)
    
    return layers.multiply([input_tensor, attention])

In [30]:
def build_spatiotemporal_autoencoder(input_shape, date_feature_dim):
    """
    Builds the core prediction model with:
    - ConvLSTM for spatiotemporal patterns
    - Date feature integration
    - Multi-feature output
    
    Args:
        input_shape: (seq_length, h, w, n_features)
        date_feature_dim: Number of date features (8)
    
    Returns:
        keras.Model: Compiled autoencoder
    """
    # Image sequence input
    image_input = layers.Input(shape=input_shape)
    
    # Encoder
    x = layers.TimeDistributed(layers.Conv2D(32, 3, activation='relu', padding='same'))(image_input)
    x = layers.TimeDistributed(spatial_attention_block)(x)
    x = layers.TimeDistributed(layers.MaxPooling2D(2, padding='same'))(x)
    
    # Temporal processing
    x = layers.ConvLSTM2D(64, 3, return_sequences=True, padding='same')(x)
    x = layers.ConvLSTM2D(128, 3, return_sequences=False, padding='same')(x)
    encoded = layers.MaxPooling2D(2, padding='same')(x)
    
    # Date processing
    date_input = layers.Input(shape=(date_feature_dim,))
    date_features = layers.Dense(64, activation='relu')(date_input)
    date_features = layers.Reshape((1, 1, 64))(date_features)
    date_features = layers.UpSampling2D(size=encoded.shape[1:3])(date_features)
    
    # Combined decoding
    combined = layers.Concatenate()([encoded, date_features])
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(combined)
    x = layers.UpSampling2D(2)(x)
    x = layers.Conv2D(input_shape[-1], 3, activation='sigmoid', padding='same')(x)
    
    return models.Model([image_input, date_input], x)

In [31]:
import re
from datetime import datetime
import os

def extract_date(filename):
    """
    Extract date from filenames like 'A19790102.png' (A + YYYYMMDD format)
    """
    try:
        basename = os.path.basename(filename)
        
        # Pattern for 'A' prefix + 8 digits (YYYYMMDD)
        match = re.search(r'A(\d{8})\.', basename)
        if match:
            date_str = match.group(1)  # Gets the 8-digit portion
            return datetime.strptime(date_str, '%Y%m%d')
        
        # Add other patterns if needed
        print(f"Warning: No date pattern matched in {filename}")
        return None
    except Exception as e:
        print(f"Error parsing date from {filename}: {str(e)}")
        return None

In [32]:
# Test with your actual filenames
test_files = ["A19790102.png", "A20211231.png", "wrong_format.txt"]
for f in test_files:
    date = extract_date(f)
    print(f"{f} → {date}")

A19790102.png → 1979-01-02 00:00:00
A20211231.png → 2021-12-31 00:00:00
wrong_format.txt → None


In [33]:
def detect_drought_areas(model, X_new, date_new, threshold_std=2.0):
    """
    Identifies drought-prone areas based on prediction anomalies.
    
    Args:
        model: Trained autoencoder
        X_new: New input sequences (n, seq_len, h, w, n_features)
        date_new: Corresponding date features
        threshold_std: Standard deviation threshold for anomalies
    
    Returns:
        tuple: (drought_mask, anomaly_scores)
    """
    y_pred = model.predict([X_new, date_new])
    
    # Compare first prediction step with last input
    error = np.abs(X_new[:, -1:] - y_pred[:, :1])
    
    # Normalize errors
    mean_err = np.mean(error, axis=(1,2,3), keepdims=True)
    std_err = np.std(error, axis=(1,2,3), keepdims=True)
    anomaly_scores = (error - mean_err) / std_err
    
    drought_mask = anomaly_scores > threshold_std
    
    return drought_mask.squeeze(), anomaly_scores.squeeze()

In [ ]:
import numpy as np
from tqdm import tqdm
import gc

# 1. First pass: Calculate total sequences and validate dates
def count_valid_sequences(filenames, seq_length=12, pred_steps=3):
    """Count how many valid sequences exist without loading all data"""
    valid_indices = []
    for i, f in enumerate(filenames):
        if extract_date(f) is not None:
            valid_indices.append(i)
    
    # Check consecutive dates (optional but recommended)
    valid_dates = [extract_date(filenames[i]) for i in valid_indices]
    date_diffs = np.diff([d.toordinal() for d in valid_dates])
    if not all(d == 1 for d in date_diffs):
        print("Warning: Non-consecutive dates found")
    
    return len(valid_indices) - seq_length - pred_steps + 1

# Get sequence count from first feature
feature = FEATURES[0]
_, filenames = load_images_from_folder(os.path.join(BASE_PATH, feature), IMG_SIZE)
total_sequences = count_valid_sequences(filenames, seq_length=12, pred_steps=3)

# 2. Preallocate arrays based on count
X_seq = np.zeros((total_sequences, 12, *IMG_SIZE, len(FEATURES)), dtype=np.float32)
y_seq = np.zeros((total_sequences, 3, *IMG_SIZE, len(FEATURES)), dtype=np.float32)
date_features = np.zeros((total_sequences, 8), dtype=np.float32)

# 3. Sequential loading with batch processing
for seq_idx in tqdm(range(total_sequences), desc="Building sequences"):
    # Calculate original time index
    time_idx = seq_idx  # Adjust if using valid_indices
    
    # Load data per-feature just-in-time
    for feat_idx, feature in enumerate(FEATURES):
        X, filenames = load_images_from_folder(os.path.join(BASE_PATH, feature), IMG_SIZE)
        
        # Fill sequences
        X_seq[seq_idx, :, :, :, feat_idx] = X[time_idx:time_idx+12]
        y_seq[seq_idx, :, :, :, feat_idx] = X[time_idx+12:time_idx+15]
    
    # Process dates
    date = extract_date(filenames[time_idx+11])  # Last date in input window
    date_features[seq_idx] = date_to_features(date)
    
    # Clear memory every 100 sequences
    if seq_idx % 100 == 0:
        gc.collect()

# 4. Verify output shapes
print(f"X_seq shape: {X_seq.shape}")  # Should be (n_seq, 12, h, w, n_feat)
print(f"y_seq shape: {y_seq.shape}")  # Should be (n_seq, 3, h, w, n_feat)

Loading images from /kaggle/input/water-scarcity/visualization_outputUp/Evap...


100%|██████████| 3308/3308 [01:53<00:00, 29.04it/s]


Building sequences:   0%|          | 0/3294 [00:00<?, ?it/s]

Loading images from /kaggle/input/water-scarcity/visualization_outputUp/Evap...



100%|██████████| 3308/3308 [01:54<00:00, 28.79it/s]


...


100%|██████████| 3308/3308 [01:46<00:00, 30.97it/s]


Loading images from /kaggle/input/water-scarcity/visualization_outputUp/SoilM_0_10cm...



100%|██████████| 3308/3308 [01:46<00:00, 31.19it/s]


Loading images from /kaggle/input/water-scarcity/visualization_outputUp/TVeg...



 98%|█████████▊| 3245/3308 [01:47<00:02, 30.12it/s]


In [ ]:
# X_all = []
# dates_all = []

# for feature in FEATURES:
#     X, filenames = load_images_from_folder(os.path.join(BASE_PATH, feature), IMG_SIZE)
#     dates = [extract_date(f) for f in filenames]
#     X_all.append(X)
#     dates_all.append(dates)

# X_combined = np.stack(X_all, axis=-1)  # (n, h, w, n_features)


In [ ]:
# Check for None values in dates
none_indices = [i for i, date in enumerate(dates_all[0]) if date is None]
print(f"Found {len(none_indices)} None dates. Examples:")
for i in none_indices[:5]:  # Print first 5 problematic files
    print(f"File: {filenames[i]}, Extracted date: {dates_all[0][i]}")

In [ ]:
strategy=setup_gpu()
with strategy.scope():
    
    model = build_spatiotemporal_autoencoder(X_seq.shape[1:], date_features.shape[1])
    model.compile(optimizer='adam', loss='mse')
    
    history = model.fit(
        [X_seq, date_features], y_seq,
        validation_split=0.2,
        epochs=50,
        batch_size=32,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(patience=5),
            tf.keras.callbacks.ModelCheckpoint('best_model.h5', save_best_only=True)
        ]
    )